LLM 就是大语言模型 Large Language Model

生成模型有三个阶段 预训练preTrain，监督微调SFT，强化学习RLHF

工作流程：io流 当我们有对应输入，会有怎样的输出
1Tokenization 分词化
    将句子分解成更小的unit，有不同的颗粒分类，不同的语言遵循不同的方式
        词粒度Word_Level Tokenization 适用于英语类似的语言
        字符粒度Character-Level 适用于中文类似的语言
        子词粒度Subword_level 适用于有词缀词根相关的语言、
工作方式
    通过给定的token预测下一个token 按照以下recursive方式去进行预测 
        1. 基于现有tokens 预测下一个最有可能的token
        2. 将得到的token加入到现有的tokens里
        3. 重复以上操作
        4. 直到输出特殊token 如句号，end of sentence，换行符号之类的

完成数据收集和token分词后，进入到模型构建部分 主要运用到的是Transformer架构

Transformer块结构：
    Input → [LayerNorm] → [Multi-Head Attention] → [Residual Add] → [LayerNorm] → [Feedforward] → [Residual Add] → Output

    1. 输入嵌入 + 位置编码
    输入嵌入就是将Token ID转换为向量
    位置编码则是注入顺序信息

    细节： 将word变化为vector的过程 叫做word2vector 每个词会按照预设的值转变为vector（这个初始值是随机的 但你要知道 你自己手动怕不是得累死 因此大伙都用之前有好人已经提供的matrix作为初始值）比如chatgpt3 它的初始word vector就有大概50257个词*12288个dimension

    而训练，就是将语义相近的词的vector尽可能的靠拢（在空间上） 因此如果直接使用训练好的word matrix，也许仅需再训练一点就能直接达成目的？

    一个简短的例子 假设function E()是word转化为vector 那么
    E(queen) - E(king) ≈ E(woman) - E(man) 

    随后是点积 dot product 这是注意力机制的根本，我们使用它作为衡量word vector的对齐程度的方法 （我认为可以理解成两个词中间的特殊关系）
    向量方向相近 dot product为正
    向量相互垂直 dot product为零
    向量方向相反 dot product为负
    举个例子 
    代表负数的plur可以通过E(cats) - E(cat)获得
    假设plur * E(cat) 那么我们就可以得到负数 因为cat是单数
    plur * E(cats) 那么我们就可以得到负数 因为cats是复数
    同样的，当我们运用在dog和dogs上，理应得到类似的结果
    有趣的是，如果我们将plur * E(one) = -2.4
    随后尝试其他数字     
    plur * E(Two) = 0.79
    plur * E(Three) = 1.27
    plur * E(Four) = 1.80
    可以看到他们是在增长的

    以上 就是初始的Embedding Matrix，总共有50257*12288个参数 我们把它叫做W_e

    但当我们将一大篇文本直接丢进去的时候，我们仅仅是将每个单词单个的从embedding matrix中翻出来罢了，他们之间没有上下文，没有情景，也没有位置关系，我们要做的，就是更新vector并使他们能够获得比单个词更丰富更具体的含义 

    但我们在处理过程中 我们的网络仅能一次处理特定数量的向量 GPT3的context size就是2048，这个大小限制了GPT的上下文长度，因此超出这个长度时，我们会发现它变得健忘



In [ ]:
    # Pseudocode
    input_embedding = embedding_layer(token_ids)  # shape: [seq_len, d_model]
    position_embedding = get_position_encoding(seq_len, d_model)
    x = input_embedding + position_embedding

    2. 多头自注意力机制
    Q = Query
    K = Key
    V = Value
    根据这三个进行计算

In [ ]:
    # Pseudocode
    Q = x @ W_q   # shape: [seq_len, d_k]
    K = x @ W_k
    V = x @ W_v
    attention_weights = softmax(Q @ K.T / sqrt(d_k))
    output = attention_weights @ V

    3. Add & LayerNorm
    将原始输入添加到注意力输出中 也就是残差连接residual connection
    规范化结果以稳定学习过程

In [ ]:
    x = LayerNorm(x + Attention(Q, K, V))

    4. 前馈网络FFN (MLP aka multiple layer perceptron 多层感知器) 两个名字都可以 取决于input和处理过程
    
    在attention之后，我们将一个简单的MLP独立的应用于每个标记的位置
    FFN(x) = ReLU(xW1 + b1)W2 + b2
    This helps model complex combinations of features per token.

    5. Final Residual + LayerNorm
    与之前相同 添加输入 然后规范化

    x = LayerNorm(x + FFN(x))

    以上就是完整的一次transformer流程，通常我们将这个过程堆积12/24/96次 每一次都利用更深层次的上下文来细化表征
    在最后一次迭代后 最终output会被传递到线性层 -> softmax over vocab
    用于在训练期间预测下一个标记

    UnEmbedding
    最后阶段，我们需要输出下一个可能的token的分布概率。
    举个例子，假如我们让这个网络了解了哈利波特这本书，并且在我们的input中包含了关键词“哈利波特”“最讨厌的”“教授” 
    那么按照我们context matrix最后一数列的那1*12288的vector，我们将它和我们所有的 12288*50257大小的矩阵W_u 也就是UnEmbedding Matrix进行映射，会得到一个1*50257大小的概率表 之后对表里每一个做softmax 最终得到每个单词的概率表 一般来说，概率最高的是概率表里数字最大的word 这个词大概率就是Snape

    为什么只选择最后一列呢？因为最后一列已经是经过前面的选择而得出的，它本身就蕴含着前面的words的信息，因此为了效率最大化，我们只选择最后一列
    
